In [ ]:
class MajorityChecker:

    def __init__(self, arr: List[int]):
        self.n = len(arr)
        self.arr = arr
        self.vals = [0] * (4 * self.n)  # maj, score
        self._build(1, 0, self.n - 1)
        self.ha = defaultdict(list)
        for i, x in enumerate(arr):
            self.ha[x].append(i)

    def push_up(self, o):
        x, xc = self.vals[o * 2]
        y, yc = self.vals[o * 2 + 1]
        if x == y: num, cnt = x, xc + yc
        elif xc > yc: num, cnt = x, xc - yc
        else: num, cnt = y, yc - xc
        self.vals[o] = (num, cnt)

    def _build(self, o, l, r):  # 初始化
        if l == r:  # basecase
            self.vals[o] = (self.arr[l], 1)
            return
        # 左右子问题 摩尔投票 得到主问题
        m = l + r >> 1
        self._build(o * 2, l, m)
        self._build(o * 2 + 1, m + 1, r)
        self.push_up(o)

    def _query(self, o, l, r, L, R):  # 查询[L,R]的val
        if r < L or l > R: return (0, 0)  # 没有交集 无关紧要值
        if L <= l and r <= R: return self.vals[o]  # 内部
        # 有交集
        m = l + r >> 1
        x, xc = self._query(o * 2, l, m, L, R)
        y, yc = self._query(o * 2 + 1, m + 1, r, L, R)
        if x == y: num, cnt = x, xc + yc
        elif xc > yc: num, cnt = x, xc - yc
        else: num, cnt = y, yc - xc
        return (num, cnt)

    def query(self, left: int, right: int, threshold: int) -> int:
        num, _ = self._query(1, 0, self.n - 1, left, right)
        pos = self.ha[num]
        d = bisect_right(pos, right) - bisect_left(pos, left)
        if d >= threshold: return num
        return -1


# Your MajorityChecker object will be instantiated and called as such:
# obj = MajorityChecker(arr)
# param_1 = obj.query(left,right,threshold)

In [ ]:
class Solution:

    def longestRepeating(self, s: str, queryCharacters: str, queryIndices: List[int]) -> List[int]:
        n = len(s)
        s = list(s)
        pre = [0] * (4 * n)  # 前缀连续字符最长长度
        suf = [0] * (4 * n)  # 后缀连续字符最长长度
        mx = [0] * (4 * n)  # 区间连续字符最长长度

        def push_up(o, l, r, m):
            pre[o] = pre[o * 2]
            suf[o] = suf[o * 2 + 1]
            mx[o] = max(mx[o * 2], mx[o * 2 + 1])
            if s[m] == s[m + 1]:
                if pre[o * 2] == m - l + 1:
                    pre[o] += pre[o * 2 + 1]
                if suf[o * 2 + 1] == r - m:
                    suf[o] += suf[o * 2]
                mx[o] = max(mx[o], suf[o * 2] + pre[o * 2 + 1])  # pre[o], suf[o]

        def build(o, l, r):
            if l == r:
                pre[o] = suf[o] = mx[o] = 1
                return
            m = l + r >> 1
            build(o * 2, l, m)
            build(o * 2 + 1, m + 1, r)
            push_up(o, l, r, m)

        def update(o, l, r, idx):  # idx设val
            if l == r:
                return
            m = l + r >> 1
            if m >= idx:
                update(o * 2, l, m, idx)
            else:
                update(o * 2 + 1, m + 1, r, idx)
            push_up(o, l, r, m)

        build(1, 0, n - 1)
        ans = []
        for i, j in enumerate(queryIndices):
            s[j] = queryCharacters[i]
            update(1, 0, n - 1, j)
            ans.append(mx[1])
        return ans